## Analysis of a quantitative chemical map

In [1]:
# import the chemMap package
import chemMap

# import common python packages for data processing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# turn off certain warnings and use the matplotlib backend: Qt5Agg to get interactive plots
pd.options.mode.chained_assignment = default='warn'
%matplotlib

# set basic plot parameters (or preferences)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 18})

Using matplotlib backend: Qt5Agg


In this example, we load in quantitative chemical map data of a plagioclase crystal from a Hawaiian gabbro collected from the flanks of Hualalai volcano. This map is collected over a ~ 3 mm by 3 mm area selected by visual inspection of a larger scale map (~ 20 μm resolution):

<img src="HawaiiQuant/106426.png">

In [2]:
Oxide = ['Fe','Mg','Ca','Ti','Al','Na','Si', 'K']
Data = chemMap.loadwt('HawaiiQuant/',Oxide,filter=True)

Resolution = 6.35

k-means cluster analysis can now be performed on this quantitative data, but before we do it is useful to convert the wt% of each element into the wt% of their respective oxides.

In [3]:
Data = chemMap.calcOxides(Data, Oxide)

We now apply the k-means cluster analysis. In this example, the user is promted to type the cluster names once the average compositions are shown.

In [4]:
Oxides = ['FeO','MgO','CaO','TiO2','Al2O3','Na2O','SiO2','K2O']
Data = chemMap.Cluster(Data, Oxides, 4, Name = True)

Cluster 0,FeO = 0.6
Cluster 0,MgO = 0.12
Cluster 0,CaO = 14.24
Cluster 0,TiO2 = 0.1
Cluster 0,Al2O3 = 32.0
Cluster 0,Na2O = 2.63
Cluster 0,SiO2 = 48.58
Cluster 0,K2O = 0.2
Cluster 0,Cluster = 0.0
Type a three letter identification code for this cluster: 
Plg
Cluster 1,FeO = 6.84
Cluster 1,MgO = 14.13
Cluster 1,CaO = 19.03
Cluster 1,TiO2 = 1.02
Cluster 1,Al2O3 = 5.7
Cluster 1,Na2O = 0.5
Cluster 1,SiO2 = 47.66
Cluster 1,K2O = -0.0
Cluster 1,Cluster = 1.0
Type a three letter identification code for this cluster: 
Cpx
Cluster 2,FeO = 2.62
Cluster 2,MgO = 0.7
Cluster 2,CaO = 2.19
Cluster 2,TiO2 = 0.1
Cluster 2,Al2O3 = 2.62
Cluster 2,Na2O = 0.18
Cluster 2,SiO2 = 4.97
Cluster 2,K2O = 0.02
Cluster 2,Cluster = 2.0
Type a three letter identification code for this cluster: 
nan
Cluster 3,FeO = 3.87
Cluster 3,MgO = 3.52
Cluster 3,CaO = 9.74
Cluster 3,TiO2 = 0.38
Cluster 3,Al2O3 = 12.03
Cluster 3,Na2O = 0.99
Cluster 3,SiO2 = 27.32
Cluster 3,K2O = 0.04
Cluster 3,Cluster = 3.0
Type a three letter ide

We can now calculate the ratio of Anorthite in the plagioclase.

In [5]:
Data = chemMap.calcRatios(Data, oxide = True, ratios = ['An', 'Mg#'])

C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:54: RuntimeWarning: invalid value encountered in true_divide
  Data[r] = (Data['CaO']/56.0774)/(Data['CaO']/56.0774+2*Data['Na2O']/61.9789)*100
C:\Users\mattg\Anaconda3\lib\site-packages\chemMap\ratios.py:51: RuntimeWarning: invalid value encountered in true_divide
  Data[r] = (Data['MgO']/40.3044)/(Data['FeO']/71.844+Data['MgO']/40.3044)*100


And create a plot of An in the plagioclase. For completeness, FeO in the pyroxene is also plotted here.

In [6]:
f, b = chemMap.ElementMap(Data, ['An','FeO'], ColMap = ['Blues','Reds'], 
                        Resolution = Resolution, Bounds = [[55, 95],[2,10]], Cluster = ['Plg','Cpx'])

This area shows a region where an exsiting gabbroic mush/cumulate is interacting with a passing melt phase. This reaction is causing reactive infiltration of the passing melt phase into the mush, leading to the observed texture.

What we may wish to do here is quantify the composition of the two different plagioclase domains. There are several ways to do this. First, we could extract a transect.

In [7]:
Tr = chemMap.Section(Data, b, 'An', Resolution, Cluster = 'Plg')

Click twice to define the start and the end of the chosen transect, and once to define the width of the area to consider


Alternatively, we could re-perform the cluster analysis, but this time only consider the Plagioclase classified pixels. This would allow us to extract the distribution of compositions for the more primitive and more evolved plagioclase regions.

In [8]:
Oxides = ['CaO','Al2O3','Na2O','SiO2']
Data = chemMap.Cluster(Data, Oxides, 3, Cluster = 'Plg', ShowComp = True, Plot = True)

Cluster_Plg 0,CaO = 0.0
Cluster_Plg 0,Al2O3 = 0.0
Cluster_Plg 0,Na2O = 0.0
Cluster_Plg 0,SiO2 = 0.0
Cluster_Plg 1,CaO = 13.6
Cluster_Plg 1,Al2O3 = 31.37
Cluster_Plg 1,Na2O = 2.99
Cluster_Plg 1,SiO2 = 49.43
Cluster_Plg 2,CaO = 15.03
Cluster_Plg 2,Al2O3 = 33.06
Cluster_Plg 2,Na2O = 2.3
Cluster_Plg 2,SiO2 = 48.29


Lets now assess the relative composition of these two plagioclase clusters

In [9]:
f, a = plt.subplots(1,1, figsize = (10,6))
a.hist(Data['An'][np.where(Data['Cluster_Plg']==1)], alpha = 0.6,
       bins  = np.linspace(45,95,201), density = True, facecolor = [1,0.2,0.2])
a.hist(Data['An'][np.where(Data['Cluster_Plg']==2)], alpha = 0.6,
       bins  = np.linspace(45,95,201), density = True, facecolor = [0.2,0.2,1])
a.set_xlabel('Anorthite Content')

Text(0.5, 0, 'Anorthite Content')